In [19]:
import requests
from bs4 import BeautifulSoup
import re
#The basics
import numpy as np
import pandas as pd
import pickle

#Get them web sites
import requests

#Make sure slenium works
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

#Start the google driver
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
#For inserting articles into Mongodb
from pymongo import MongoClient

In [20]:
mj_first_articles = pickle.load(open("pickle_files/mj_first_articles.pkl", "rb" ))
mj_more_articles = pickle.load(open("pickle_files/mj_more_articles.pkl", "rb" ))

In [21]:


def get_article(url):
#Sends request for url
    html = requests.get(url).text
    return html

def parse_article(html):
#BeautifulSoups the article
    soup = BeautifulSoup(html, 'lxml')
    title = soup.find('h1', class_='entry-title').text
    body = soup.find('article', class_='entry-content').text

    article = {
        'title': title,
        'body': body,
        'source': 'Mother Jones',
        'num_source': 9
    }

    return article

def get_parsed_article_from_link(url):
#Runs the previous two functions on the url
    return parse_article(get_article(url))

In [22]:
#phase 1: uses request to try to BeautifulSoup links
mj_list_o_articles = []
mj_problem_articles = []
for text in mj_first_articles:
    #print(text)
    try:
        art = get_parsed_article_from_link(text.encode())
        mj_list_o_articles.append(art)

    except:
        print("Problem processing url " + text)
        problem = text
        mj_problem_articles.append(problem)
    time.sleep(2)

In [23]:
#phase 2: uses selenium to go through links
driver = webdriver.Chrome(chromedriver)
for x in mj_problem_articles:
    try:
        driver.get(x)
        time.sleep(3)
        art = get_parsed_article_from_link(text.encode())
        mj_list_o_articles.append(art)
    except:
        pass

In [24]:
#Same as above but for expanded list, phase 1
mj_more_list_o_articles = []
mj_more_problem_articles = []
for text in mj_more_articles:
    #print(text)
    try:
        art = get_parsed_article_from_link(text.encode())
        mj_more_list_o_articles.append(art)

    except:
        print("Problem processing url " + text)
        problem = text
        mj_more_problem_articles.append(problem)
    time.sleep(2)


In [25]:
#same as above for expanded list, phase 2
driver = webdriver.Chrome(chromedriver)
for x in mj_more_problem_articles:
    try:
        driver.get(x)
        time.sleep(3)
        art = get_parsed_article_from_link(text.encode())
        mj_more_list_o_articles.append(art)
    except:
        pass

In [26]:
all_articles = mj_list_o_articles + mj_more_list_o_articles

In [27]:
len(all_articles)

25

In [28]:
#starts client in Mongodb
client = MongoClient()
test = client.derp.test

In [29]:
#creates event and loads articles into Mongodb
db = client.events
test = db.test
test.insert_many(all_articles)


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

In [ ]:
test.count()